In [21]:
import pandas as pd
import numpy as np
import random as rd

from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn import svm

import warnings 
warnings.filterwarnings('ignore')

In [2]:
dfOrderDict = pd.read_excel('ENB2012_data.xls', sheetname=None)
dfOrderDict.keys()

odict_keys(['Φύλλο1', 'Φύλλο2', 'Φύλλο3'])

In [3]:
#for rename key, dfOrderDict['cek3'] = dfOrderDict.pop('Φύλλο3')
data = dfOrderDict['Φύλλο1']

In [4]:
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [5]:
#shuffle the data
data = data.sample(frac=1)
X1 = data.iloc[:,:-2]
Y = data['Y1']

#get dummies var to X6&X8 (categorical)
Xbef = pd.get_dummies(X1,columns=['X6','X8'])

#Feature Scaling - MinMax
min_max_scaler=preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(Xbef)

# GENETIC ALGORITHM

In [6]:
Cnt1 = len(X)

p_c = 1
p_m = 0.2
pop = 100
gen = 50
kfold = 3

In [7]:
#the first 15 would represent C params
#the next 13 would represent gamma params
XY0 = np.array([1,1,1,1,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,1,1,1,1,0,0,1,0,1,0])

#empty list, in order to have the population to populate it with the population
n_list = np.empty((0,len(XY0)))
n_list.shape

(0, 30)

In [8]:
#shuffle XY0 and stack them on top of each other
#so that we can have a random population

for i in range(pop):
    rd.shuffle(XY0) #untuk ngerandom XY0 
    n_list = np.vstack((n_list,XY0)) #ngerandom XY0 sebanyak (pop)x dimasukin ke n_list
    
#check n_list
n_list

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 1., 0.],
       ...,
       [1., 1., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 0., 1., 0.]])

In [9]:
n_list.shape

(100, 30)

In [10]:
#Calculate the FV
#C
a_X = 10
b_X = 1000
l_X = (len(XY0)/2) #HALF LENGTH OF CROMOSOMS. 
#each population have 30 cromosoms, 15 for C, 15 for gamma

#gamma
a_Y = 0.05
b_Y = 0.99
l_Y = (len(XY0)/2)

#precision
Precision_X = (b_X - a_X)/((2**l_X)-1)
Precision_Y = (b_Y - a_Y)/((2**l_Y)-1)

### Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
menggunakan XY0(1,30) BUKAN n_list(100,30)

In [11]:
# Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
z = 0
t = 1 #index dari belakang, (-)1,(-)2,
X0_num_Sum = 0

for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
    X0_num = XY0[-t]*2**z 
    #indexing XY0[-t], mulai dari belakang, (-1,-2,-3 dst)
    #2 pangkat z, mulai dari 0,
    X0_num_Sum += X0_num #jumlahin semua hasil perkaliannya
    
    t = t+1
    z = z+1

In [12]:
# Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
p = 0
u = 1+(len(XY0)//2) 
#len(XY0)/2 = 15. tapi, indexing start from (-16,  karna . yg X start from -1 bukan 0)
Y0_num_Sum = 0

for i in range(len(XY0)//2):
    Y0_num = XY0[-u]*2**z
    Y0_num_Sum += Y0_num
    
    u = u+1
    p = p+1

## DECODING C & GAMMA

In [13]:
DecodedX = (X0_num_Sum*Precision_X)+a_X
DecodedY = (Y0_num_Sum*Precision_Y)+a_Y

print('DecodedX:',DecodedX)
print('DecodedY:',DecodedY)

DecodedX: 730.5877864925078
DecodedY: 9.450286873989075


In [14]:
n_list.shape

(100, 30)

In [15]:
np.vstack((n_list,XY0))

array([[0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 1., 0.],
       ...,
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.]])

## empty list for backup data

In [16]:
Final_Best_in_Generation_X = []
Worst_Best_in_Generation_X = []

One_Final_Guy = np.empty((0,len(XY0)+2))
One_Final_Guy_Final = []

Min_for_all_Generations_for_Mut_1 = np.empty((0,len(XY0)+1))
Min_for_all_Generations_for_Mut_2 = np.empty((0,len(XY0)+1))

Min_for_all_Generations_for_Mut_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2 = np.empty((0,len(XY0)+2))

Min_for_all_Generations_for_Mut_1_1_1 = np.empty((0,len(XY0)+2))
Min_for_all_Generations_for_Mut_2_2_2 = np.empty((0,len(XY0)+2))

In [1]:
Generation = 1 

for i in range(gen):
    New_Population = np.empty((0,len(XY0)))
    
    All_in_Generation_X_1 = np.empty((0,len(XY0)+1))
    All_in_Generation_X_2 = np.empty((0,len(XY0)+1))
    
    Min_in_Generation_X_1 = []
    Min_in_Generation_X_2 = []
    
    Save_Best_in_Generation_X = np.empty((0,len(XY0)))+1
    Final_Best_in_Generation_X = []
    Worst_Best_in_Generation_X = []
    
    #print()
    #print("GENERATION: #", Generation)
#----------------------------------------------------------------------------------------------------------
    Family = 1 

    for j in range(int(pop/2)):
    #     print()
    #     print("Family:#:", Family)

        #TOURNAMENT SELECTION
        Parents = np.empty((0,len(XY0)))

        for i in range(2): #LOOP 2X UNTUK DAPETIN 2 PARENTS

            Battle_Troops = []

            #recording to tournament selection, we need to choose 3 contestant 
            #where they will fight and see who is the best one will picked up
            #1 angka, randomly, nomor index dari panjangnya n_list(100)
            Warrior_1_index = np.random.randint(0,len(n_list))
            Warrior_2_index = np.random.randint(0,len(n_list))
            Warrior_3_index = np.random.randint(0,len(n_list))

            while Warrior_1_index == Warrior_2_index:
                Warrior_1_index = np.random.randint(0,len(n_list))

            while Warrior_2_index == Warrior_3_index:
                Warrior_2_index = np.random.randint(0,len(n_list))

            while Warrior_1_index == Warrior_3_index:
                Warrior_1_index = np.random.randint(0,len(n_list))

            Warrior_1 = n_list[Warrior_1_index]
            Warrior_2 = n_list[Warrior_2_index]
            Warrior_3 = n_list[Warrior_3_index]

            Battle_Troops = [Warrior_1,Warrior_2,Warrior_3]

            #----------------------------------------------------------------------------------------------------------
            ## Warrior1
            #Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
            z = 0
            t = 1 #index dari belakang, (-)1,(-)2,
            X0_num_Sum_W1 = 0

            for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
                X0_num_W1 = Warrior_1[-t]*2**z 
                X0_num_Sum_W1 += X0_num_W1 #jumlahin semua hasil perkaliannya
                t = t+1
                z = z+1

            #WARRIOR 1,GAMMA 
            Y0_num_Sum = 0
            Y0_num_Sum_W1 = 0

            p = 0
            u = 1+(len(XY0)//2) 

            for i in range(len(XY0)//2):
                Y0_num_W1 = Warrior_1[-u]*2**z
                Y0_num_Sum_W1 += Y0_num_W1

                u = u+1
                p = p+1


            #DECODING
            Decoded_X_W1 = (X0_num_Sum_W1*Precision_X)+a_X
            Decoded_Y_W1 = (Y0_num_Sum_W1*Precision_Y)+a_Y

            P_1 = 0

            from sklearn.model_selection import KFold
            kf = KFold(n_splits=kfold)
            kf = kf.split(X)
            for train_index, test_index in kf:    
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVR(kernel='rbf', C=Decoded_X_W1, gamma=Decoded_Y_W1)
                model1.fit(X_train,Y_train)
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test) #GET THE ACCURACY

                OF_So_Far_1 = 1-(AC1) #CHANGE ACCURACY INTO ERROR
                P_1 += OF_So_Far_1

            OF_So_Far_W1=P_1/kfold
            # OF_So_Far_W1= AKAN MENJADI RATA-RATA DARI NILAI ERROR DARI TIAP ITERASI KFOLD

            #----------------------------------------------------------------------------------------------------------
            ## Warrior2
            #Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
            z = 0
            t = 1 #index dari belakang, (-)1,(-)2,
            X0_num_Sum_W2 = 0

            for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
                X0_num_W2 = Warrior_2[-t]*2**z 
                X0_num_Sum_W2 += X0_num_W2 #jumlahin semua hasil perkaliannya
                t = t+1
                z = z+1

            #WARRIOR 1,GAMMA 
            Y0_num_Sum = 0
            Y0_num_Sum_W2 = 0

            p = 0
            u = 1+(len(XY0)//2) 

            for i in range(len(XY0)//2):
                Y0_num_W2 = Warrior_1[-u]*2**z
                Y0_num_Sum_W2 += Y0_num_W2

                u = u+1
                p = p+1


            #DECODING
            Decoded_X_W2 = (X0_num_Sum_W2*Precision_X)+a_X
            Decoded_Y_W2 = (Y0_num_Sum_W2*Precision_Y)+a_Y

            P_1 = 0

            from sklearn.model_selection import KFold
            kf = KFold(n_splits=kfold)
            kf = kf.split(X)
            for train_index, test_index in kf:    
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVR(kernel='rbf', C=Decoded_X_W2, gamma=Decoded_Y_W2)
                model1.fit(X_train,Y_train)
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test) #GET THE ACCURACY

                OF_So_Far_1 = 1-(AC1) #CHANGE ACCURACY INTO ERROR
                P_1 += OF_So_Far_1

            OF_So_Far_W2=P_1/kfold
            #----------------------------------------------------------------------------------------------------------
            ## Warrior3
            #Jumlah perkalian index dengan (0/1) untuk tiap kromosom pada 1 populasi
            z = 0
            t = 1 #index dari belakang, (-)1,(-)2,
            X0_num_Sum_W3 = 0

            for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
                X0_num_W3 = Warrior_2[-t]*2**z 
                X0_num_Sum_W3 += X0_num_W3 #jumlahin semua hasil perkaliannya
                t = t+1
                z = z+1

            #WARRIOR 1,GAMMA 
            Y0_num_Sum = 0
            Y0_num_Sum_W3 = 0

            p = 0
            u = 1+(len(XY0)//2) 

            for i in range(len(XY0)//2):
                Y0_num_W3 = Warrior_1[-u]*2**z
                Y0_num_Sum_W3 += Y0_num_W3

                u = u+1
                p = p+1


            #DECODING
            Decoded_X_W3 = (X0_num_Sum_W3*Precision_X)+a_X
            Decoded_Y_W3 = (Y0_num_Sum_W3*Precision_Y)+a_Y

            P_1 = 0

            from sklearn.model_selection import KFold
            kf = KFold(n_splits=kfold)
            kf = kf.split(X)
            for train_index, test_index in kf:    
                X_train,X_test = X[train_index],X[test_index]
                Y_train,Y_test = Y[train_index],Y[test_index]

                model1 = svm.SVR(kernel='rbf', C=Decoded_X_W3, gamma=Decoded_Y_W3)
                model1.fit(X_train,Y_train)
                PL1 = model1.predict(X_test)

                AC1 = model1.score(X_test,Y_test) #GET THE ACCURACY

                OF_So_Far_1 = 1-(AC1) #CHANGE ACCURACY INTO ERROR
                P_1 += OF_So_Far_1

            OF_So_Far_W3=P_1/kfold

            #----------------------------------------------------------------------------------------------------------
            Prize_Warrior_1 = OF_So_Far_W1
            Prize_Warrior_2 = OF_So_Far_W2
            Prize_Warrior_3 = OF_So_Far_W3

            if Prize_Warrior_1 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_1
                Prize = Prize_Warrior_1

            if Prize_Warrior_2 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_2
                Prize = Prize_Warrior_2

            if Prize_Warrior_3 == min(Prize_Warrior_1,Prize_Warrior_2,Prize_Warrior_3):
                Winner = Warrior_3
                Prize = Prize_Warrior_3

            Parents = np.vstack((Parents,Winner))
        Parent_1 = Parents[0]
        Parent_2 = Parents[1]
        
        #print(Parent_1,'\n--------')
        #print(Parent_2)
        
        #EMPTY ARRAY FOR CHILDREN - crossover
        Child_1 = np.empty((0,len(XY0)))
        Child_2 = np.empty((0,len(XY0)))
        
        Ran_CO_1 = np.random.rand() #RandomCrossOver1
        
        if Ran_CO_1 < p_c: #where p_c selalu 1 (diset diawal)
            
            Cr_1 = np.random.randint(0,len(XY0))
            Cr_1 = np.random.randint(0,len(XY0))
            
            while Cr_1 == Cr_2: #kalo sama maka pilih ulang
                Cr_2 = np.random.randint(0,len(XY0))
            
            #pop = 4 , gen=1, set to get faster process   
            if Cr_1 <Cr_2: #Cr like threshold/batasan

                Med_Seg_1 = Parent_1[Cr_1:Cr_2+1]
                Med_Seg_2 = Parent_2[Cr_1:Cr_2+1]
                
                First_Seg_1 = Parent_1[:Cr_1]
                Sec_Seg_1 = Parent_1[Cr_2+1:]
                
                First_Seg_2 = Parent_2[:Cr_1]
                Sec_Seg_2 = Parent_2[Cr_2+1:]
                
                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))
            else:

                Med_Seg_1 = Parent_1[Cr_2:Cr_1+1]
                Med_Seg_2 = Parent_2[Cr_2:Cr_1+1]
                
                First_Seg_1 = Parent_1[:Cr_2]
                Sec_Seg_1 = Parent_1[Cr_1+1:]
                
                First_Seg_2 = Parent_2[:Cr_2]
                Sec_Seg_2 = Parent_2[Cr_1+1:]
                
                Child_1 = np.concatenate((First_Seg_1,Med_Seg_2,Sec_Seg_1))
                Child_2 = np.concatenate((First_Seg_2,Med_Seg_1,Sec_Seg_2))
                
        else:  #No crossOver happen
            Child_1 = Parent_1
            Child_2 = Parent_2
            
            
        #----------------------------------------------------------------------------------------------------------    
        Mutated_Child_1 = []
        
        t=0
        
        for i in Child_1:
            
            Ran_Mut_1 = np.random.rand()
            
            #p_m=probabilitas mutation
            if Ran_Mut_1 < p_m:
                if Child_1[t] == 0:
                    Child_1[t] = 1
                else:
                    Child_1[t] = . 0
                t = t+1
                
                Mutated_Child_1 = Child_1
            else:
                Mutated_Child_1 = Child_1
                
        #----------------------------------------------------------------------------------------------------------            
        Mutated_Child_2 = []
        
        t=0
        
        for i in Child_2:
            
            Ran_Mut_2 = np.random.rand()
            
            #p_m=probabilitas mutation
            if Ran_Mut_2 < p_m:
                if Child_2[t] == 0:
                    Child_2[t] = 1
                else:
                    Child_2[t] = . 0
                t = t+1
                
                Mutated_Child_2 = Child_2
            else:
                Mutated_Child_2 = Child_2
        
        #----------------------------------------------------------------------------------------------------------            
        #DECODE THE MUTATED CHILD 1
        
        #C - Mutated_Child_1------------------------------
        X0_num_Sum_MC_1 = 0
        
        z = 0
        t = 1 #index dari belakang, (-)1,(-)2,

        for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
            X0_num_M1 = Mutated_Child_1[-t]*2**z 
            X0_num_Sum_MC_1 += X0_num_M1 #jumlahin semua hasil perkaliannya
            t = t+1
            z = z+1

        #GAMMA - Mutated_Child_1------------------------------
        Y0_num_Sum_MC_1 = 0
        p = 0
        u = 1+(len(XY0)//2) 

        for i in range(len(XY0)//2):
            Y0_num_M1 = Mutated_Child_1[-u]*2**z
            Y0_num_Sum_MC_1 += Y0_num_M1

            u = u+1
            p = p+1


        #DECODING------------------------------
        Decoded_X_M1 = (X0_num_Sum_MC_1*Precision_X)+a_X
        Decoded_Y_M1 = (Y0_num_Sum_MC_1*Precision_Y)+a_Y

        P_1 = 0

        from sklearn.model_selection import KFold
        kf = KFold(n_splits=kfold)
        kf = kf.split(X)
        for train_index, test_index in kf:    
            X_train,X_test = X[train_index],X[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVR(kernel='rbf', C=Decoded_X_M1, gamma=Decoded_Y_M1)
            model1.fit(X_train,Y_train)
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test) #GET THE ACCURACY

            OF_So_Far_1 = 1-(AC1) #CHANGE ACCURACY INTO ERROR
            P_1 += OF_So_Far_1

        OF_So_Far_M1=P_1/kfold
        #----------------------------------------------------------------------------------------------------------
        #DECODE THE MUTATED CHILD 2
        
        #C - Mutated_Child_2------------------------------
        X0_num_Sum_MC_2 = 0
        
        z = 0
        t = 1 #index dari belakang, (-)1,(-)2,

        for i in range(len(XY0)//2): #setengah nya . karna dibagi 2)
            X0_num_M2 = Mutated_Child_2[-t]*2**z 
            X0_num_Sum_MC_2 += X0_num_M2 #jumlahin semua hasil perkaliannya
            t = t+1
            z = z+1

        #GAMMA - Mutated_Child_2------------------------------
        Y0_num_Sum_MC_2 = 0
        p = 0
        u = 1+(len(XY0)//2) 

        for i in range(len(XY0)//2):
            Y0_num_M2 = Mutated_Child_2[-u]*2**z
            Y0_num_Sum_MC_2 += Y0_num_M2

            u = u+1
            p = p+1


        #DECODING------------------------------
        Decoded_X_M2 = (X0_num_Sum_MC_2*Precision_X)+a_X
        Decoded_Y_M2 = (Y0_num_Sum_MC_2*Precision_Y)+a_Y

        P_1 = 0

        from sklearn.model_selection import KFold
        kf = KFold(n_splits=kfold)
        kf = kf.split(X)
        for train_index, test_index in kf:    
            X_train,X_test = X[train_index],X[test_index]
            Y_train,Y_test = Y[train_index],Y[test_index]

            model1 = svm.SVR(kernel='rbf', C=Decoded_X_M2, gamma=Decoded_Y_M2)
            model1.fit(X_train,Y_train)
            PL1 = model1.predict(X_test)

            AC1 = model1.score(X_test,Y_test) #GET THE ACCURACY

            OF_So_Far_1 = 1-(AC1) #CHANGE ACCURACY INTO ERROR
            P_1 += OF_So_Far_1

        OF_So_Far_M2=P_1/kfold
        #----------------------------------------------------------------------------------------------------------
        print()
        print("FV for . Mutated Child #1 at Gen", Generation,":",OF_So_Far_M1)
        print("FV for . Mutated Child #2 at Gen", Generation,":",OF_So_Far_M2)
        
        #Stack them on the top of each other, so we can keep track of the mutant children
        All_in_Generation_X_1_1_temp = Mutated_Child_1[np.newaxis]  #Switch the axis, it will be vertical, we need to make . it horizontal
        All_in_Generation_X_1_1 = np.column_stack((OF_So_Far_M1, All_in_Generation_X_1_1_temp))  
        
        All_in_Generation_X_2_2_temp = Mutated_Child_2[np.newaxis]  #Switch the axis, it will be vertical, we need to make . it horizontal
        All_in_Generation_X_2_2 = np.column_stack((OF_So_Far_M2, All_in_Generation_X_2_2_temp))  
        
        All_in_Generation_X_1 = np.vstack((All_in_Generation_X_1,All_in_Generation_X_1_1))
        All_in_Generation_X_2 = np.vstack((All_in_Generation_X_2,All_in_Generation_X_2_2))
        
        Save_Best_in_Generation_X = np.vstack((All_in_Generation_X_1,All_in_Generation_X_2))
        #Save_Best : an array with all those the mutated children
        
        New_Population = np.vstack((New_Population,Mutated_Child_1,Mutated_Child_2))
        #----------------------------------------------------------------------------------------------------------
        R_1 = []
        t = 0
        for i in All_in_Generation_X_1:
            
            if(All_in_Generation_X_1[t,:1]) <= min(All_in_Generation_X_1[:,:1]):
                R_1 = All_in_Generation_X_1[t,:]
            t = t+1
            
        
        Min_in_Generation_X_1 = R_1[np.newaxis]
        
        
        R_2 = []
        t = 0
        for i in All_in_Generation_X_2:
            
            if(All_in_Generation_X_2[t,:1]) <= min(All_in_Generation_X_2[:,:1]):
                R_2 = All_in_Generation_X_2[t,:]
            t = t+1
            
        
        Min_in_Generation_X_2 = R_2[np.newaxis]
        Family = Family+1
        #----------------------------------------------------------------------------------------------------------

    t = 0
    R_11 = []
    for i in Save_Best_in_Generation_X:
        
        if (Save_Best_in_Generation_X[t,:1]) <= min(Save_Best_in_Generation_X[:,:1]):
            R_11 = Save_Best_in_Generation_X[t,:]
        t = t+1
            
    Final_Best_in_Generation_X = R_11[np.newaxis]
    
    
    t = 0
    R_22 = []
    for i in Save_Best_in_Generation_X:
        
        if (Save_Best_in_Generation_X[t,:1]) >= max(Save_Best_in_Generation_X[:,:1]):
            R_22 = Save_Best_in_Generation_X[t,:1]
        t = t+1
            
    Worst_Best_in_Generation_X = R_22[np.newaxis]
    
    
    Darwin_Guy = Final_Best_in_Generation_X[:]
    Not_So_Darwin_Guy = Worst_Best_in_Generation_X[:]
    
    
    Darwin_Guy = Darwin_Guy[0:,1:].tolist()
    Not_So_Darwin_Guy = Not_So_Darwin_Guy[0:,1:].tolist()
    
    Best_1 = np.where((New_Population==Darwin_Guy))
    Worst_1 = np.where((New_Population==Not_So_Darwin_Guy))
    
    
    New_Population[Worst_1] = Darwin_Guy
    
    n_list = New_Population

    
    Min_for_all_Generations_for_Mut_1 = np.vstack((Min_for_all_Generations_for_Mut_1,
                                                   Min_in_Generation_X_1))
    Min_for_all_Generations_for_Mut_2 = np.vstack((Min_for_all_Generations_for_Mut_2,
                                                   Min_in_Generation_X_2))
    
    
    Min_for_all_Generations_for_Mut_1_1 = np.insert(Min_in_Generation_X_1,0,Generation)
    Min_for_all_Generations_for_Mut_2_2 = np.insert(Min_in_Generation_X_2,0,Generation)
    
    
    Min_for_all_Generations_for_Mut_1_1_1 = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                                                       Min_for_all_Generations_for_Mut_1_1))
    
    Min_for_all_Generations_for_Mut_2_2_2 = np.vstack((Min_for_all_Generations_for_Mut_2_2_2,
                                                       Min_for_all_Generations_for_Mut_2_2))
    
    
    Generation = Generation+1
    
    
    
One_Final_Guy = np.vstack((Min_for_all_Generations_for_Mut_1_1_1,
                           Min_for_all_Generations_for_Mut_2_2_2))


t = 0
Final_Here = []

for i in One_Final_Guy:
    if(One_Final_Guy[t,1]) <= min(One_Final_Guy[:,1]):
        Final_Here = One_Final_Guy[t,:]
    t = t+1

One_Final_Guy_Final = Final_Here[np.newaxis]

print()
print("Min in all Generations:",One_Final_Guy_Final)

print()
print("Final Solution:",One_Final_Guy_Final[:,2:])
print("Highest R2:",(1-One_Final_Guy_Final[:,1]))

XY0_Encoded_After = One_Final_Guy_Final[:,2:][0]


z = 0
t = 1
X0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    X0_num_Encoded_After = XY0_Encoded_After[-t]*2**z
    X0_num_Sum_Encoded_After += X0_num_Encoded_After
    t = t+1
    z = z+1
    
p = 0
u = 1+(len(XY0)//2)
Y0_num_Sum_Encoded_After = 0

for i in range(len(XY0)//2):
    Y0_num_Encoded_After = XY0_Encoded_After[-u]*2**p
    Y0_num_Sum_Encoded_After += Y0_num_Encoded_After
    u = u+1
    p = p+1


Decoded_X_Encoded_After = (X0_num_Sum_Encoded_After*Precision_X)+a_X
Decoded_Y_Encoded_After = (Y0_num_Sum_Encoded_After*Precision_Y)+a_Y

print()
print("Decoded_X_Encoded_After:",Decoded_X_Encoded_After)
print("Decoded_Y_Encoded_After:",Decoded_Y_Encoded_After)
        